In [1]:
import pandas as pd
import geopandas as gpd

from shapely.geometry import Point

In [2]:
samples = pd.read_parquet('samples.pq')

df_columns = list(samples.columns)

df_columns.append('continent')

In [3]:
# Converter o DataFrame para um GeoDataFrame
geometry_samples = [Point(xy) for xy in zip(samples['x'], samples['y'])]

gdf_samples = gpd.GeoDataFrame(samples, geometry=geometry_samples, crs="EPSG:4326")

In [4]:
# Carregar o shapefile com fronteiras dos continentes
continents = gpd.read_file("world_continents/World_Continents.shp")

In [5]:
# Certificar que ambos os GeoDataFrames estão no mesmo CRS
gdf_samples = gdf_samples.to_crs(continents.crs)

In [6]:
# Realizar o Spatial Join
result_samples = gpd.sjoin(gdf_samples, continents, how="left", predicate="intersects")

result_train = result_samples.rename(columns={'CONTINENT': 'continent'})

result_train = result_train[df_columns]

In [8]:
result_train.to_parquet("c_samples.pq")